In [9]:
import pandas as pd

dataset = pd.read_csv('chunked_datav2.csv')

dataset

,page_id,page_title,chunk_id,chunk_text,category
0,0,Présentation – FST,0,Présentation – FST - LA FACULTÉ I- PRESENTATIO...,Faculty
1,0,Présentation – FST,1,"filières LST, 13 filières MST, 4 filières Ingé...",Faculty
2,0,Présentation – FST,2,Les Mathématiques et Informatique Autres Fabla...,Faculty
3,0,Présentation – FST,3,urbaine. Le Green Lab est ouvert aux étudiants...,Faculty
4,1,Mot du Doyen – FST,4,Mot du Doyen – FST - Le monde d’aujourd’hui co...,Other
...,...,...,...,...,...
140,66,Club Enactus – FST,140,Club Enactus – FST - L’équipe ENACTUS de la Fa...,Activity
141,66,Club Enactus – FST,141,"– Former des leaders de demain, – Participer a...",Activity
142,67,Club Biotechnologie – FST,142,Club Biotechnologie – FST - Le Club Biotechnol...,Activity
143,68,Future Leaders – FST,143,Future Leaders – FST - Are you the future lead...,Other


In [10]:
from pinecone import Pinecone
import os

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY") or "your pinecone api key"

# configure client
pc = Pinecone(api_key=api_key)

In [11]:
index_name = 'chatbot'

index = pc.Index(index_name)

In [12]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 183}},
 'total_vector_count': 183}

In [5]:
#from langchain.embeddings import HuggingFaceInstructEmbeddings

#embedding_model = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

In [13]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\Dell\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = [embeddings.embed_query(text) for text in texts]
len(res), len(res[0])

(2, 384)

In [15]:
res

[[-0.011914805509150028,
  0.0891803726553917,
  0.0382428914308548,
  0.019081171602010727,
  0.05977565050125122,
  0.005316327791661024,
  0.03878144547343254,
  -0.008798966184258461,
  0.0602349117398262,
  -0.015470278449356556,
  0.027178650721907616,
  0.058878201991319656,
  -0.025384072214365005,
  -0.03383246809244156,
  -0.01912357285618782,
  -0.0002414058690192178,
  0.06661204993724823,
  -0.06140350550413132,
  -0.004256864544004202,
  -0.00203145993873477,
  0.027551129460334778,
  0.13663719594478607,
  0.0045463927090168,
  0.02143852971494198,
  0.04543565586209297,
  0.07764166593551636,
  -0.09471824765205383,
  0.06313017010688782,
  0.06228695809841156,
  -0.03511491045355797,
  -0.006000120658427477,
  -0.029994642361998558,
  0.15588082373142242,
  0.05987440049648285,
  0.016393357887864113,
  0.04131563752889633,
  0.008261485025286674,
  0.028696304187178612,
  0.0257016159594059,
  0.051181063055992126,
  0.03644575551152229,
  -0.11936520785093307,
  -0.0

In [18]:
from tqdm.auto import tqdm  # for progress bar

data = pd.read_csv('chunked_datav2.csv')

batch_size = 10

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{x['page_id']}-{x['chunk_id']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [x['chunk_text'] for _, x in batch.iterrows()]
    # embed text
    embeds = embeddings.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk_text'],
         'category': x['category'],
         'title': x['page_title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

  0%|          | 0/15 [00:00<?, ?it/s]

In [19]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 145}},
 'total_vector_count': 145}

In [20]:
print(dir(index))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api_client', '_config', '_iter_dataframe', '_parse_non_empty_args', '_parse_sparse_values_arg', '_upsert_batch', '_vector_api', 'delete', 'describe_index_stats', 'fetch', 'list', 'list_paginated', 'query', 'update', 'upsert', 'upsert_from_dataframe']


In [13]:
#def get_similiar_docs(query,k=1,score=False):
 # if score:
  #  similar_docs = index.similarity_search_with_score(query,k=k)
  #else:
  #  similar_docs = index.similarity_search(query,k=k)
  #return similar_docs


#query = "faculte"
#similar_docs = get_similiar_docs(query)
#similar_docs

In [22]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)

In [26]:
query = "club"

vectorstore.similarity_search(query, k=3)

[Document(page_content="Clubs – FST - GREENOLOGY CADAC CLUB GÉNIE CIVIL CLUB LES SOPHISTES Design & Photography CLUB D'ASTRONOMIE نادي القرآن الكريم Club Echec & Mat Club Enactus Club Géoinformation Club Biotechnologie Future Leaders", metadata={'category': 'Activity', 'title': 'Clubs – FST'}),
 Document(page_content='Club Génie civil – FST - Le Club Génie civil est un club scientifique de la faculté des sciences et techniques Tanger crée en 2014 par l’étudiant KALOUCH Hammam Qui vise a développer la culture scientifiques chez les étudiants de la FSTT ainsi que présenter la filière LST et MST Génie Civil de la FSTT et d’organiser plusieurs activité scientifiques dans le domaine de Génie civil, qui incluent des visites de chantiers, des conférences, des Ateliers et Un évènement annuel la journée Génie civil FSTT . le Parrain du club Est Mr Mustapha El Metoui Professeur en Génie civil, du département des physiques Président du club : KALOUCH Hammam, étudiant en 3eme année LST Génie civil